In [1]:
# Run in terminal!
# vllm serve "ConfidentialMind/Mistral-Small-24B-Instruct-2501_GPTQ_G32_W4A16"

In [2]:
import sys
print(sys.version)

3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 18:08:17) [GCC 12.2.0]


In [3]:
from openai import OpenAI
import os
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

2025-04-18 13:26:31.384619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744997191.793921  138638 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744997191.857993  138638 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744997192.772947  138638 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744997192.772988  138638 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744997192.772990  138638 computation_placer.cc:177] computation placer alr

In [4]:
# Load BLIP-2
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# Use the smaller BLIP-2 model with OPT
model_id = "Salesforce/blip2-opt-2.7b"
processor = Blip2Processor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Load image
image = Image.open("jeans.png").convert("RGB")

# Use instruction-style prompt as a QUESTION (not declarative)
prompt = "What is the person wearing? Describe color, fabric, and style."

inputs = processor(image, prompt, return_tensors="pt").to("cpu")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Process
#

# Generate
with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,
        temperature=0.7
    )

# Decode
caption = processor.decode(generated_ids[0], skip_special_tokens=True)
print("🧵 Caption:", caption)

/home/arg8qqv/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


🧵 Caption: What is the person wearing? Describe color, fabric, and style.



In [8]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

model_id = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_id)
model = BlipForConditionalGeneration.from_pretrained(model_id).to("cpu")  # force CPU

image = Image.open("jeans.png").convert("RGB")
prompt = ""  # <-- this is key

inputs = processor(image, prompt, return_tensors="pt").to("cpu")
outputs = model.generate(**inputs, max_length=50)
caption = processor.decode(outputs[0], skip_special_tokens=True)

print("Caption:", caption)

Caption: a woman in a white top and blue pants


In [9]:
# Light-weight BLIP model (needs ~2GB GPU)
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

model_id = "Salesforce/blip-image-captioning-base"  # or blip-image-captioning-large
processor = BlipProcessor.from_pretrained(model_id)
model = BlipForConditionalGeneration.from_pretrained(model_id).to("cpu")#to("cuda" if torch.cuda.is_available() else "cpu")

image = Image.open("jeans.png").convert("RGB")
prompt = "Describe the primary piece of clothing. Describe colors, patterns, fabrics, and styles."

inputs = processor(image, prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=50)
print(processor.decode(outputs[0], skip_special_tokens=True))

describe the primary piece of clothing. describe colors, patterns, fabrics, and styles.s


In [10]:
# Point to your local VLLM server
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="sk-fake-key"  # Any string works; vLLM doesn’t check this
)

In [11]:
message = f'Return just the description of the article of clothing: {caption}. Return no other text.'

response = client.chat.completions.create(
    model="ConfidentialMind/Mistral-Small-24B-Instruct-2501_GPTQ_G32_W4A16",
    messages=[
        {"role": "user", "content": message}
    ],
    temperature=0.15
)

# Print the response
print(response.choices[0].message.content)

a white top and blue pants


In [12]:
import pandas as pd

#df = pd.read_csv("data/pinterest_trends.csv")
#df.head()
with open("data/pinterest_trends.csv", "r", encoding="utf-8", errors="ignore") as f:
    for i in range(50):
        print(f"{i}: {f.readline().strip()}")


0: "Pinterest Trends tool – https://trends.pinterest.com/?l1InterestIds=948967005229%7C924581335376&amp;sortOrder=desc&amp;sortBy=MONTHLY_CHANGE"
1: Selected Filters
2: Trends Type,Growing trends
3: Date Range,90 days before 2025-04-07
4: Interests,"Women's Fashion,Men's Fashion"
5: Include keywords,"All"
6: Age,"All"
7: Gender,"All"
8: 
9: ,,,,,Data in the date columns reflects the normalized search volume for this trend type
10: Rank,Trend,Weekly change,Monthly change,Yearly change,2025-01-13,2025-01-20,2025-01-27,2025-02-03,2025-02-10,2025-02-17,2025-02-24,2025-03-03,2025-03-10,2025-03-17,2025-03-24,2025-03-31,2025-04-07
11: 1,graduation outfit college,"-6%","2,000%","200%",1,1,1,1,2,2,2,6,15,77,88,100,94
12: 2,cute summer bitmoji outfits,"10%","400%","-40%",6,8,7,7,14,9,8,23,40,82,73,90,100
13: 3,spring style 2025,"2%","400%","10,000%+",2,3,5,6,7,7,14,23,33,55,73,97,100
14: 4,easter outfit women,"50%","400%","-80%",2,3,3,4,6,9,16,20,29,91,49,66,100
15: 5,spring dinner outfit,"-20%"

In [13]:
df = pd.read_csv(
    "data/pinterest_trends.csv",
    skiprows=10,
    engine="python",
    encoding="utf-8",
    on_bad_lines='warn'
)
df.head()

Skipping line 16: Expected 18 fields in line 16, saw 30
Skipping line 18: Expected 18 fields in line 18, saw 30
Skipping line 20: Expected 18 fields in line 20, saw 30
Skipping line 21: Expected 18 fields in line 21, saw 31
Skipping line 24: Expected 18 fields in line 24, saw 31
Skipping line 26: Expected 18 fields in line 26, saw 31
Skipping line 27: Expected 18 fields in line 27, saw 30
Skipping line 28: Expected 18 fields in line 28, saw 31
Skipping line 29: Expected 18 fields in line 29, saw 28
Skipping line 32: Expected 18 fields in line 32, saw 31
Skipping line 33: Expected 18 fields in line 33, saw 31
Skipping line 34: Expected 18 fields in line 34, saw 31
Skipping line 36: Expected 18 fields in line 36, saw 31
Skipping line 37: Expected 18 fields in line 37, saw 31
Skipping line 38: Expected 18 fields in line 38, saw 31
Skipping line 40: Expected 18 fields in line 40, saw 31
Skipping line 41: Expected 18 fields in line 41, saw 31
Skipping line 42: Expected 18 fields in line 42,

,Rank,Trend,Weekly change,Monthly change,Yearly change,2025-01-13,2025-01-20,2025-01-27,2025-02-03,2025-02-10,2025-02-17,2025-02-24,2025-03-03,2025-03-10,2025-03-17,2025-03-24,2025-03-31,2025-04-07
0,1,graduation outfit college,-6%,"2,000%",200%,1,1,1,1,2,2,2,6,15,77,88,100,94
1,2,cute summer bitmoji outfits,10%,400%,-40%,6,8,7,7,14,9,8,23,40,82,73,90,100
2,3,spring style 2025,2%,400%,"10,000%+",2,3,5,6,7,7,14,23,33,55,73,97,100
3,4,easter outfit women,50%,400%,-80%,2,3,3,4,6,9,16,20,29,91,49,66,100
4,6,lululemon outfit fashion,2%,300%,"1,000%",19,12,12,16,10,7,5,29,74,86,83,97,100


In [14]:
import json

# Sanitize column names
df.columns = df.columns.map(str).str.strip()
df = df.rename(columns={df.columns[1]: "Trend"})

# Build prompt/response pairs
def make_prompt(row):
    return f"Describe the trend: {row['Trend']}"

def make_output(row):
    return (
        f"Weekly change: {row['Weekly change']}, "
        f"Monthly change: {row['Monthly change']}, "
        f"Yearly change: {row['Yearly change']}."
    )

# Format dataset
data = []
for _, row in df.iterrows():
    if pd.notnull(row['Trend']):
        data.append({
            "input": make_prompt(row),
            "output": make_output(row)
        })

# Save to JSONL
with open("pinterest_finetune.jsonl", "w") as f:
    for item in data:
        f.write(json.dumps(item) + "\n")

print(f"✅ Saved {len(data)} training examples!")

✅ Saved 19 training examples!


In [15]:
with open("pinterest_finetune.jsonl", "w") as f:
    for item in df:
        f.write(json.dumps(item) + "\n")

In [16]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")


# Load and tokenize dataset
dataset = load_dataset("json", data_files="pinterest_finetune.jsonl")["train"]

def tokenize(example):
    prompt = f"### Instruction:\n{example['input']}\n\n### Response:\n{example['output']}"
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

dataset = dataset.map(tokenize)

# LoRA config
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05
)

model = get_peft_model(model, peft_config)

# Training args
training_args = TrainingArguments(
    output_dir="./mistral-pinterest-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_total_limit=2,
    logging_steps=10,
    bf16=True,
    optim="paged_adamw_8bit"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
401 Client Error. (Request ID: Root=1-68028edb-7a7c5367113b6bf44906ce24;c85200f4-10e3-49b5-80d6-595578c5898b)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [25]:
caption = "jean shorts"  # ← From BLIP

# Optional: add trend context from the Pinterest dataset
trend_examples = df["Trend"].dropna().sample(5).tolist()
trend_context = ", ".join(trend_examples)

# Build prompt
prompt = f"""
This outfit was detected in a photo: "{caption}".
Based on current Pinterest fashion trends — like {trend_context} — analyze whether this outfit is trendy or not, and explain why.
""".strip()

# Send to Mistral
response = client.chat.completions.create(
    model="ConfidentialMind/Mistral-Small-24B-Instruct-2501_GPTQ_G32_W4A16",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7
)

print("✨", response.choices[0].message.content)


✨ To determine whether "jean shorts" are trendy based on the current fashion trends mentioned, we need to consider the overall trends and how they might relate to "jean shorts."

### Current Fashion trends:
1. **Red plaid shorts outfit**: This trend involves plaid patterns, which are popular in various fashion trends.
2. **Plus size cowboy boots outfit**: This trend focuses on cowboy boots, which are often associated with a rustic or western style.
3. **Girly style outfits**: This trend involves feminine styles that are often characterized by soft, delicate, and feminine designs.
4. **Platform wedge sandals**: This trend involves sandals with platform wedges, which are trendy for their unique design.
5. **Spring style 2025**: This trend generally focuses on bright, fresh, and spring-like designs.

### Analysis of "jean shorts":
- **Jean shorts** are typically associated with casual and comfortable styles. They are often worn in summer and spring seasons.
- Jean shorts are generally ass